In [1]:
import pandas as pd

In [2]:
ls

Data Cleaning.ipynb       README.md
Data Visualization.ipynb  zippedData/


In [3]:
cd zippedData/

/Users/josephmchugh/code/flatiron-project-1/zippedData


In [4]:
#importing datasets
movie_gross = pd.read_csv('bom.movie_gross.csv.gz')
name_basics = pd.read_csv('imdb.name.basics.csv.gz')
title_akas = pd.read_csv('imdb.title.akas.csv.gz')
title_basics = pd.read_csv('imdb.title.basics.csv.gz')
title_crew = pd.read_csv('imdb.title.crew.csv.gz')
title_principles = pd.read_csv('imdb.title.principals.csv.gz')
title_ratings = pd.read_csv('imdb.title.ratings.csv.gz')
rt_movie_info = pd.read_csv('rt.movie_info.tsv.gz', sep='\t')
rt_reviews = pd.read_csv('rt.reviews.tsv.gz', sep='\t', encoding = 'unicode_escape')
tmdb_movies = pd.read_csv('tmdb.movies.csv.gz')
movie_budgets = pd.read_csv('tn.movie_budgets.csv.gz')

In [5]:
#creating main dataframe
main_df = pd.DataFrame(movie_budgets['movie'])

In [6]:
#populating main dataframe from 'movie_budgets' dataset
main_df['Production Budget'] = movie_budgets['production_budget']
main_df['Domestic Gross'] = movie_budgets['domestic_gross']
main_df['Worldwide Gross'] = movie_budgets['worldwide_gross']

In [7]:
#renaming columns
main_df = main_df.rename(columns={'movie':'Title'})
title_basics = title_basics.rename(columns={'primary_title': 'Title'})

In [8]:
#merging 'title_baiscs' dataset with main dataframe
main_df = pd.merge(main_df, title_basics, how='outer', on='Title')

In [9]:
#trimming size of dataframe to first 5782 rows (number of rows with financial data)
main_df = main_df.head(5782)

In [10]:
#dropping uneccessary columns
main_df = main_df.drop(columns = ['tconst','original_title','start_year'])

In [11]:
#dropping null values from 'genres column'
main_df = main_df.dropna(subset=['genres'])

In [12]:
#resetting the index
main_df = main_df.reset_index()

In [13]:
#dropping old index 
main_df = main_df.drop(columns = ['index'])

In [14]:
main_df

,Title,Production Budget,Domestic Gross,Worldwide Gross,runtime_minutes,genres
0,Avatar,"$425,000,000","$760,507,625","$2,776,345,279",93.0,Horror
1,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",136.0,"Action,Adventure,Fantasy"
2,Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",113.0,"Action,Adventure,Sci-Fi"
3,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",141.0,"Action,Adventure,Sci-Fi"
4,Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,134,200",149.0,"Action,Adventure,Sci-Fi"
...,...,...,...,...,...,...
2979,Truth or Dare,"$3,500,000","$41,411,015","$95,127,344",90.0,"Drama,Romance"
2980,Truth or Dare,"$3,500,000","$41,411,015","$95,127,344",100.0,"Horror,Thriller"
2981,Truth or Dare,"$3,500,000","$41,411,015","$95,127,344",92.0,"Comedy,Drama,Romance"
2982,Friday,"$3,500,000","$27,467,564","$27,936,778",117.0,Family


In [15]:
#removing '$' and ',' from financial data
main_df["Production Budget"] = main_df["Production Budget"].apply(lambda x: x.strip("$"))
main_df["Production Budget"] = main_df["Production Budget"].apply(lambda x: x.replace(",",""))
main_df["Domestic Gross"] = main_df["Domestic Gross"].apply(lambda x: x.strip("$"))
main_df["Domestic Gross"] = main_df["Domestic Gross"].apply(lambda x: x.replace(",",""))
main_df["Worldwide Gross"] = main_df["Worldwide Gross"].apply(lambda x: x.strip("$"))
main_df["Worldwide Gross"] = main_df["Worldwide Gross"].apply(lambda x: x.replace(",",""))

In [16]:
#converting financial data to type(int)
main_df["Production Budget"] = main_df["Production Budget"].astype(int)
main_df["Domestic Gross"] = main_df["Domestic Gross"].astype(int)
main_df["Worldwide Gross"] = main_df["Worldwide Gross"].astype(int)

In [17]:
#correcting Avatar runtime & genre
main_df.loc[(main_df['Title'] == 'Avatar'), ['runtime_minutes']] = 162
main_df.loc[(main_df['Title'] == 'Avatar'), ['genres']] = 'Action, Adventure, Sci-Fi'

In [18]:
main_df["Profit"] = main_df["Worldwide Gross"] - main_df["Production Budget"] 

In [19]:
#Adding commas to numbers for clarity
# main_df['Production Budget'] = main_df['Production Budget'].apply(lambda x: format(x, '8,d'))
# main_df['Domestic Gross'] = main_df['Domestic Gross'].apply(lambda x: format(x, '8,d'))
# # main_df['Worldwide Gross'] = main_df['Worldwide Gross'].apply(lambda x: format(x, '8,d'))
# main_df['Profit'] = main_df['Profit'].apply(lambda x: format(x, '8,d'))

In [20]:
#removing title duplicates
main_df = main_df.drop_duplicates(subset="Title", keep="first")

In [21]:
main_df["genres"].nunique() #count of unique genres

274

In [22]:
main_df = main_df.sort_values(by=['Profit'], ascending = False)

In [31]:
main_df.head(10)

,Title,Production Budget,Domestic Gross,Worldwide Gross,runtime_minutes,genres,Profit
0,Avatar,425000000,760507625,2776345279,162.0,"Action, Adventure, Sci-Fi",2351345279
40,Titanic,200000000,659363944,2208208395,NaN,Adventure,2008208395
4,Avengers: Infinity War,300000000,678815482,2048134200,149.0,"Action,Adventure,Sci-Fi",1748134200
25,Jurassic World,215000000,652270625,1648854864,124.0,"Action,Adventure,Sci-Fi",1433854864
65,Furious 7,190000000,353007020,1518722794,137.0,"Action,Crime,Thriller",1328722794
19,The Avengers,225000000,623279547,1517935897,143.0,"Action,Adventure,Sci-Fi",1292935897
39,Black Panther,200000000,700059566,1348258224,134.0,"Action,Adventure,Sci-Fi",1148258224
111,Jurassic World: Fallen Kingdom,170000000,417719760,1305772799,128.0,"Action,Adventure,Sci-Fi",1135772799
151,Frozen,150000000,400738009,1272469910,93.0,"Adventure,Drama,Sport",1122469910
130,Beauty and the Beast,160000000,504014165,1259199706,112.0,"Drama,Fantasy,Romance",1099199706


In [32]:
main_df1 = main_df.loc[(main_df['Profit'] >= 100000000)] #with only 3 days to clean and process the data so we want to focus on movies that generates more than 10mil profit 

In [42]:
len(main_df1['genres'].unique())

126

In [43]:
genres_profit = main_df.groupby(main_df['genres']).mean()

In [45]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
genres_profit

,Production Budget,Domestic Gross,Worldwide Gross,runtime_minutes,Profit
genres,,,,,
Action,4.651250e+07,3.782093e+07,7.658565e+07,93.571429,3.007315e+07
"Action, Adventure, Sci-Fi",4.250000e+08,7.605076e+08,2.776345e+09,162.000000,2.351345e+09
"Action,Adventure",7.462500e+07,6.842080e+07,1.664437e+08,110.000000,9.181869e+07
"Action,Adventure,Animation",1.151471e+08,1.567252e+08,4.374047e+08,100.294118,3.222576e+08
"Action,Adventure,Biography",8.000000e+07,6.303523e+07,2.206284e+08,137.500000,1.406284e+08
"Action,Adventure,Comedy",9.101786e+07,1.232708e+08,3.385101e+08,111.111111,2.474922e+08
"Action,Adventure,Crime",4.816667e+07,4.561213e+07,1.329990e+08,103.833333,8.483228e+07
"Action,Adventure,Drama",8.835185e+07,5.859742e+07,1.690086e+08,116.481481,8.065675e+07
"Action,Adventure,Family",1.215000e+08,9.172938e+07,2.559627e+08,108.500000,1.344627e+08


In [27]:
genres_profit = genres_profit.sort_values(by=['Profit'], ascending = False)

In [28]:
genre_counts = main_df['genres'].value_counts()

In [30]:
genre_counts

Drama                         136
Adventure,Animation,Comedy     67
Comedy                         63
Comedy,Drama                   61
Action,Adventure,Sci-Fi        53
                             ... 
Crime,Horror,Thriller           1
Action,Crime,Sport              1
Comedy,Sport                    1
Action,Family,Fantasy           1
Comedy,Drama,Sport              1
Name: genres, Length: 274, dtype: int64

In [ ]:
main_df1 #we are treating worldwide gross as revenue (although it is only one component if revenue, aka the direct sales to retailer and consumers. also we are treating production budget as total expense when there is also marketing budget..ect. )

In [ ]:
#can create a histogram
#set unique words = 0 then iterate =+1
#use contains, startwith, endwith

In [ ]:
main_df1= main_df = main_df.dropna(subset=['runtime_minutes'])

In [ ]:
main_df1

In [ ]:
main_df.dropna() #removing null values row to keep our data consistent

In [ ]:
# main_df1 = main_df1.groupby('genres')

In [ ]:
# print(main_df1)

In [ ]:
main_

In [ ]:
grouped_df = main_df1.groupby(['genres']).mean()

In [ ]:
grouped_df.sort_values(by=['Profit'], ascending = False)

In [ ]:
#how to remove columns with negative values